In [34]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from skimage.measure import compare_psnr,compare_mse
from scipy import ndimage, misc
from random import shuffle
import math
from keras.models import model_from_json, load_model
from keras.layers import *
from keras.models import Model
import re
from keras import regularizers
from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger, EarlyStopping, ModelCheckpoint
from keras.layers.core import Lambda
from keras.preprocessing.image import ImageDataGenerator
from keras.losses import mean_squared_error


import pickle
from random import randint
from utilities import *
%matplotlib inline


In [3]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,:,0], rgb[:,:,:,1], rgb[:,:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append(img)
    return np.array(images)
        

In [35]:
def atoi(text) : 
    return int(text) if text.isdigit() else text
def natural_keys(text) :
    return [atoi(c) for c in re.split('(\d+)', text)]


root_path = ""
filenames = []

for root, dirnames, filenames in os.walk("./data/oct_gt_resized/"):
    filenames.sort(key = natural_keys)
    rootpath = root
ground_truth_rs = []
for filename in filenames :
    filepath = os.path.join(root,filename)
    image = ndimage.imread(filepath, mode = "L")
    ground_truth_rs.append(image)
    print(filename)


root_path = ""
filenames = []

for root, dirnames, filenames in os.walk("./data/oct_noisy_resized/"):
    filenames.sort(key = natural_keys)
    rootpath = root
noisy_images_rs = []
for filename in filenames :
    filepath = os.path.join(root,filename)
    image = ndimage.imread(filepath, mode = "L")
    noisy_images_rs.append(image)
    print(filename)

   
ground_truth_rs = np.array(ground_truth_rs)
ground_truth_rs.shape
ground_truth_rs = np.resize(ground_truth_rs,(ground_truth_rs.shape[0],ground_truth_rs.shape[1],ground_truth_rs.shape[2],1))
ground_truth_rs.shape

0.tif
1.tif
2.tif
3.tif
4.tif
5.tif
6.tif
7.tif
8.tif
9.tif
10.tif
11.tif
12.tif
13.tif
14.tif
15.tif
16.tif
17.tif
18.tif
19.tif
20.tif
21.tif
22.tif
23.tif
24.tif
25.tif
26.tif
27.tif
28.tif
29.tif
30.tif
31.tif
32.tif
33.tif
34.tif
35.tif
36.tif
37.tif
38.tif
39.tif
40.tif
41.tif
42.tif
43.tif
44.tif
45.tif
46.tif
47.tif
48.tif
49.tif
50.tif
51.tif
52.tif
53.tif
54.tif
55.tif
56.tif
57.tif
58.tif
59.tif
60.tif
61.tif
62.tif
63.tif
64.tif
65.tif
66.tif
67.tif
68.tif
69.tif
70.tif
71.tif
72.tif
73.tif
74.tif
75.tif
76.tif
77.tif
78.tif
79.tif
80.tif
81.tif
82.tif
83.tif
84.tif
85.tif
86.tif
87.tif
88.tif
89.tif
90.tif
91.tif
92.tif
93.tif
94.tif
95.tif
96.tif
97.tif
98.tif
99.tif
100.tif
101.tif
102.tif
103.tif
104.tif
105.tif
106.tif
107.tif
108.tif
109.tif
110.tif
111.tif
112.tif
113.tif
114.tif
115.tif
116.tif
117.tif
118.tif
119.tif
120.tif
121.tif
122.tif
123.tif
124.tif
125.tif
126.tif
127.tif
128.tif
129.tif
130.tif
131.tif
132.tif
133.tif
134.tif
135.tif
136.tif
137.tif
138.ti

250.tif
251.tif
252.tif
253.tif
254.tif
255.tif
256.tif
257.tif
258.tif
259.tif
260.tif
261.tif
262.tif
263.tif
264.tif
265.tif
266.tif
267.tif
268.tif
269.tif
270.tif
271.tif
272.tif
273.tif
274.tif
275.tif
276.tif
277.tif
278.tif
279.tif
280.tif
281.tif
282.tif
283.tif
284.tif
285.tif
286.tif
287.tif
288.tif
289.tif
290.tif
291.tif
292.tif
293.tif
294.tif
295.tif
296.tif
297.tif
298.tif
299.tif
300.tif
301.tif
302.tif
303.tif
304.tif
305.tif
306.tif
307.tif
308.tif
309.tif
310.tif
311.tif
312.tif
313.tif
314.tif
315.tif
316.tif
317.tif
318.tif
319.tif
320.tif
321.tif
322.tif
323.tif
324.tif
325.tif
326.tif
327.tif
328.tif
329.tif
330.tif
331.tif
332.tif
333.tif
334.tif
335.tif
336.tif
337.tif
338.tif
339.tif
340.tif
341.tif
342.tif
343.tif
344.tif
345.tif
346.tif
347.tif
348.tif
349.tif
350.tif
351.tif
352.tif
353.tif
354.tif
355.tif
356.tif
357.tif
358.tif
359.tif
360.tif
361.tif
362.tif
363.tif
364.tif
365.tif
366.tif
367.tif
368.tif
369.tif
370.tif
371.tif
372.tif
373.tif
374.tif


(833, 128, 256, 1)

In [3]:
from PIL import Image

In [4]:
def resize_images(src_path,dest_path,sizeX,sizeY):
    images = []
    width,height = sizeX,sizeY
    for filename in os.listdir(src_path):
        img = cv2.imread(src_path + filename)
        if img is not None:
            images.append(img)
    cnt =1
    for img in images:
        im2 = cv2.resize(img,(width, height),interpolation = cv2.INTER_CUBIC)
        
        
        ext = ".png"
        cv2.imwrite(dest_path + "image" +  str(cnt) + ext,im2)
        cnt+=1

In [5]:
raw_images = read_images_from_folder('./data/oct_gt/')
raw_images = np.resize(raw_images,(raw_images.shape[0],448,900,1))

In [6]:
#Adding noise to images
def add_noise(images):
    
    noisy_set = []
    for image in images:
        print len(noisy_set)
        for shape in xrange(2,9,1):
            for scale in xrange(2,9,1):
                noise = np.random.gamma(shape,scale,image.shape)
                noisy_set.append((image,np.clip(noise + image,0.,255.)))
    shuffle(noisy_set)
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def expand_pair(noisy_set):   
    ground_truth=[]
    noisy_images = []
    for i in range(noisy_set.shape[0]):
        ground_truth.append(noisy_set[i][0].reshape((noisy_set[i][0].shape[0],noisy_set[i][0].shape[1],1)))
        #print( str(noisy_set[i][0].shape[0]) +" "+ str(noisy_set[i][0].shape[1]))
        noisy_images.append(noisy_set[i][1].reshape((noisy_set[i][1].shape[0],noisy_set[i][1].shape[1],1)))
    return np.array(ground_truth), np.array(noisy_images)

In [7]:
noisy_set = add_noise(raw_images)
for i in range(noisy_set.shape[0]):
    cv2.imwrite("./data/oct_noisy/"+str(i)+".tif",noisy_set[i][1])    

0
49
98
147
196
245
294
343
392
441
490
539
588
637
686
735
784


In [8]:
#Shuffling and adding noise to the dataset
ground_truth,noisy_images = expand_pair(noisy_set)
print (ground_truth.shape, noisy_images.shape, noisy_set.shape)

((833, 448, 900, 1), (833, 448, 900, 1), (833, 2, 448, 900, 1))


In [9]:
print np.std(ground_truth/255.)

0.125008434593


In [10]:
def resize_images(images,size):
    resized_images = []
    
    for img in images:
        im2 = cv2.resize(img,size,interpolation = cv2.INTER_CUBIC)
        resized_images.append(im2)
    
    resized_images = np.array(resized_images)
    resized_images = np.reshape(resized_images,(resized_images.shape[0],resized_images.shape[1],resized_images.shape[2],1))
    
    return resized_images

ground_truth_rs = resize_images(ground_truth,(256,128))
noisy_images_rs = resize_images(noisy_images,(256,128))

In [11]:
noisy_images_rs.shape

(833, 128, 256, 1)

In [12]:
for i in range(noisy_images_rs.shape[0]):
    cv2.imwrite("./data/oct_noisy_resized/"+str(i)+".tif",noisy_images_rs[i])  
    
for i in range(ground_truth_rs.shape[0]):
    cv2.imwrite("./data/oct_gt_resized/"+str(i)+".tif",ground_truth_rs[i])    

In [4]:
ground_truth_rs = read_images_from_folder('./data/oct_gt_resized/')
ground_truth_rs = np.resize(ground_truth_rs,(ground_truth_rs.shape[0],ground_truth_rs.shape[1],ground_truth_rs.shape[2],1))

noisy_images_rs = read_images_from_folder('./data/oct_noisy_resized/')
noisy_images_rs = np.resize(noisy_images_rs,(noisy_images_rs.shape[0],noisy_images_rs.shape[1],noisy_images_rs.shape[2],1))

In [17]:
cv2.imwrite('test_noisy.png',noisy_images_rs[33])
cv2.imwrite('test.png',ground_truth_rs[33])

True

In [5]:
#Split into training and cross validation and normalizing
train_size = int(ground_truth_rs.shape[0]*0.8)
x_train = ground_truth_rs[0:train_size]/255.
x_train_noisy = noisy_images_rs[0:train_size]/255.
x_test = ground_truth_rs[train_size:]/255.
x_test_noisy = noisy_images_rs[train_size:]/255.
print (x_train_noisy.shape)
print (x_test_noisy.shape)

(666, 128, 256, 1)
(167, 128, 256, 1)


In [6]:
#lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1), cooldown=0, patience=2, min_lr=0.5e-6)
#csv_logger = CSVLogger('./models/simple_cnn_autoencoder.csv')
#early_stopper = EarlyStopping(min_delta=0.001,patience=30)
model_checkpoint = ModelCheckpoint('./models/simple_cnn_oct.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [18]:
x_train_noisy.shape

(666, 128, 256, 1)

In [67]:
#Defining the model

def get_simple_cnn_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=((None,None,1)))  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = AveragePooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = AveragePooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        
        
        autoencoder.compile(optimizer='Adam', loss='binary_crossentropy',metrics = ['accuracy','mean_squared_error'])

    print (autoencoder.summary())
    return autoencoder

In [69]:
#Training the model
autoencoder = get_simple_cnn_autoencoder_model()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 1)     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, None, None, 64)    640       
_________________________________________________________________
average_pooling2d_5 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
average_pooling2d_6 (Average (None, None, None, 64)    0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, None, None, 64)    36928     
_________________________________________________________________
up_sampling2d_9 (UpSampling2 (None, None, None, 64)    0         
__________

In [78]:
plot_model(autoencoder, to_file='model.png',show_shapes='True',show_layer_names='True')

In [39]:
autoencoder.fit(x_train_noisy, x_train,
                epochs=500,
                batch_size=20,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True), model_checkpoint])

Train on 666 samples, validate on 167 samples
Epoch 1/500
666/666 [==============================] - 53s - loss: 0.6267 - acc: 0.0000e+00 - mean_squared_error: 0.0120 - val_loss: 0.6170 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0063
Epoch 2/500
666/666 [==============================] - 48s - loss: 0.6117 - acc: 0.0000e+00 - mean_squared_error: 0.0052 - val_loss: 0.6140 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0049
Epoch 3/500
666/666 [==============================] - 48s - loss: 0.6090 - acc: 0.0000e+00 - mean_squared_error: 0.0039 - val_loss: 0.6176 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0065
Epoch 4/500
666/666 [==============================] - 48s - loss: 0.6071 - acc: 0.0000e+00 - mean_squared_error: 0.0030 - val_loss: 0.6149 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0053
Epoch 5/500
666/666 [==============================] - 48s - loss: 0.6061 - acc: 0.0000e+00 - mean_squared_error: 0.0026 - val_loss: 0.6094 - val_acc: 0.0000e+00 - val_mean_s

666/666 [==============================] - 48s - loss: 0.6028 - acc: 0.0000e+00 - mean_squared_error: 0.0011 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0011
Epoch 45/500
666/666 [==============================] - 48s - loss: 0.6029 - acc: 0.0000e+00 - mean_squared_error: 0.0011 - val_loss: 0.6060 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0012
Epoch 46/500
666/666 [==============================] - 48s - loss: 0.6030 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0012
Epoch 47/500
666/666 [==============================] - 48s - loss: 0.6030 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0011
Epoch 48/500
666/666 [==============================] - 48s - loss: 0.6031 - acc: 0.0000e+00 - mean_squared_error: 0.0012 - val_loss: 0.6063 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0014
Epoch 49/500
666/666 [===========

666/666 [==============================] - 48s - loss: 0.6025 - acc: 0.0000e+00 - mean_squared_error: 9.7271e-04 - val_loss: 0.6055 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0010
Epoch 67/500
666/666 [==============================] - 48s - loss: 0.6027 - acc: 0.0000e+00 - mean_squared_error: 0.0011 - val_loss: 0.6054 - val_acc: 0.0000e+00 - val_mean_squared_error: 9.6456e-04
Epoch 68/500
666/666 [==============================] - 48s - loss: 0.6024 - acc: 0.0000e+00 - mean_squared_error: 9.2018e-04 - val_loss: 0.6056 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0010
Epoch 69/500
666/666 [==============================] - 48s - loss: 0.6024 - acc: 0.0000e+00 - mean_squared_error: 9.4691e-04 - val_loss: 0.6058 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0011
Epoch 70/500
666/666 [==============================] - 48s - loss: 0.6025 - acc: 0.0000e+00 - mean_squared_error: 9.6363e-04 - val_loss: 0.6055 - val_acc: 0.0000e+00 - val_mean_squared_error: 0.0010
Epoch 71/500


666/666 [==============================] - 48s - loss: 0.6021 - acc: 0.0000e+00 - mean_squared_error: 7.8756e-04 - val_loss: 0.6054 - val_acc: 0.0000e+00 - val_mean_squared_error: 9.6467e-04
Epoch 111/500
666/666 [==============================] - 48s - loss: 0.6023 - acc: 0.0000e+00 - mean_squared_error: 8.7461e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.2725e-04
Epoch 112/500
666/666 [==============================] - 48s - loss: 0.6022 - acc: 0.0000e+00 - mean_squared_error: 8.2663e-04 - val_loss: 0.6053 - val_acc: 0.0000e+00 - val_mean_squared_error: 9.4702e-04
Epoch 113/500
666/666 [==============================] - 48s - loss: 0.6021 - acc: 0.0000e+00 - mean_squared_error: 7.9698e-04 - val_loss: 0.6051 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.4674e-04
Epoch 114/500
666/666 [==============================] - 48s - loss: 0.6024 - acc: 0.0000e+00 - mean_squared_error: 9.5032e-04 - val_loss: 0.6053 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6019 - acc: 0.0000e+00 - mean_squared_error: 7.3195e-04 - val_loss: 0.6050 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.7728e-04
Epoch 156/500
666/666 [==============================] - 48s - loss: 0.6020 - acc: 0.0000e+00 - mean_squared_error: 7.6145e-04 - val_loss: 0.6048 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.3207e-04
Epoch 157/500
666/666 [==============================] - 48s - loss: 0.6021 - acc: 0.0000e+00 - mean_squared_error: 7.9718e-04 - val_loss: 0.6054 - val_acc: 0.0000e+00 - val_mean_squared_error: 9.8277e-04
Epoch 158/500
666/666 [==============================] - 48s - loss: 0.6020 - acc: 0.0000e+00 - mean_squared_error: 7.5837e-04 - val_loss: 0.6048 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.2143e-04
Epoch 159/500
666/666 [==============================] - 48s - loss: 0.6019 - acc: 0.0000e+00 - mean_squared_error: 7.1642e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 

...

666/666 [==============================] - 48s - loss: 0.6019 - acc: 0.0000e+00 - mean_squared_error: 7.0170e-04 - val_loss: 0.6050 - val_acc: 0.0000e+00 - val_mean_squared_error: 8.0237e-04
Epoch 313/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.4609e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.4052e-04
Epoch 314/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2024e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.4618e-04
Epoch 315/500
666/666 [==============================] - 48s - loss: 0.6018 - acc: 0.0000e+00 - mean_squared_error: 6.5820e-04 - val_loss: 0.6050 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.8727e-04
Epoch 316/500
666/666 [==============================] - 48s - loss: 0.6018 - acc: 0.0000e+00 - mean_squared_error: 6.5805e-04 - val_loss: 0.6050 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.1434e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3591e-04
Epoch 358/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.3036e-04 - val_loss: 0.6048 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.3380e-04
Epoch 359/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2499e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.5146e-04
Epoch 360/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2068e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.4043e-04
Epoch 361/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.1591e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.3690e-04 - val_loss: 0.6048 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.1137e-04
Epoch 403/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2100e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.2862e-04
Epoch 404/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2494e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3283e-04
Epoch 405/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0190e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.1692e-04
Epoch 406/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0027e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.9780e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.8982e-04
Epoch 448/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0615e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.5683e-04
Epoch 449/500
666/666 [==============================] - 48s - loss: 0.6018 - acc: 0.0000e+00 - mean_squared_error: 6.7082e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 7.4135e-04
Epoch 450/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.1743e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.3384e-04
Epoch 451/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2551e-04 - val_loss: 0.6049 - val_acc: 0.0000e+00 - val_mean_squared_error: 

666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 6.0320e-04 - val_loss: 0.6045 - val_acc: 0.0000e+00 - val_mean_squared_error: 5.9663e-04
Epoch 493/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.7459e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.2028e-04
Epoch 494/500
666/666 [==============================] - 48s - loss: 0.6017 - acc: 0.0000e+00 - mean_squared_error: 6.2781e-04 - val_loss: 0.6047 - val_acc: 0.0000e+00 - val_mean_squared_error: 6.7466e-04
Epoch 495/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.9491e-04 - val_loss: 0.6045 - val_acc: 0.0000e+00 - val_mean_squared_error: 5.9256e-04
Epoch 496/500
666/666 [==============================] - 48s - loss: 0.6016 - acc: 0.0000e+00 - mean_squared_error: 5.8406e-04 - val_loss: 0.6046 - val_acc: 0.0000e+00 - val_mean_squared_error: 

In [47]:
from keras_contrib.losses import DSSIMObjective
import keras_contrib.backend as KC

#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None
    
def ms_ssim_loss(y_true,y_pred):
        print y_true.shape,y_pred.shape
        kernel = [3,3]
        y_true = KC.reshape(y_true, [-1] + list(self.__int_shape(y_pred)[1:]))
        y_pred = KC.reshape(y_pred, [-1] + list(self.__int_shape(y_pred)[1:]))

        patches_pred = KC.extract_image_patches(y_pred, kernel, kernel, 'valid', self.dim_ordering)
        patches_true = KC.extract_image_patches(y_true, kernel, kernel, 'valid', self.dim_ordering)

        # Reshape to get the var in the cells
        bs, w, h, c1, c2, c3 = self.__int_shape(patches_pred)
        patches_pred = KC.reshape(patches_pred, [-1, w, h, c1 * c2 * c3])
        patches_true = KC.reshape(patches_true, [-1, w, h, c1 * c2 * c3])
        # Get mean
        u_true = KC.mean(patches_true, axis=-1)
        u_pred = KC.mean(patches_pred, axis=-1)
        # Get variance
        var_true = K.var(patches_true, axis=-1)
        var_pred = K.var(patches_pred, axis=-1)
        # Get std dev
        covar_true_pred = K.mean(patches_true * patches_pred, axis=-1) - u_true * u_pred

        ssim = (2 * u_true * u_pred + self.c1) * (2 * covar_true_pred + self.c2)
        denom = (K.square(u_true) + K.square(u_pred) + self.c1) * (var_pred + var_true + self.c2)
        ssim /= denom  # no need for clipping, c1 and c2 make the denom non-zero
        return K.mean((1.0 - ssim) / 2.0)

In [48]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    #gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x, factor : factor * x, arguments = {'factor': gatePercentageFactor})(inputLayer)
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(None,None,1), name = "Image_input")  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        fg1,c1 = get_gated_connections(0.1,x1)
        
        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(fg3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(fg4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model(input_img,layers)
        sym_autoencoder.compile(optimizer='adam', loss = ms_ssim_loss, metrics = ['accuracy',ms_ssim_loss])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [49]:
sym_autoencoder = get_cnn_dsc_architecture()

(?, ?, ?, ?) (?, ?, ?, 1)


NameError: global name 'self' is not defined

In [21]:
plot_model(sym_autoencoder, to_file='sym_model1.png',show_shapes='True',show_layer_names='True')

In [27]:
model_checkpoint1 = ModelCheckpoint('./models/gated_cnn_autoencoder_oct_5layers_relu_adam_500E_8B_loss_msssim.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [28]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs = 500,
                batch_size = 8,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True),model_checkpoint1])

Train on 666 samples, validate on 167 samples
Epoch 1/500


InvalidArgumentError: computed output size would be negative
	 [[Node: loss_4/conv2d_transpose_30_loss/Conv2D_22 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/cpu:0"](loss_4/conv2d_transpose_30_loss/mul_47, loss_4/conv2d_transpose_30_loss/div_17)]]

Caused by op u'loss_4/conv2d_transpose_30_loss/Conv2D_22', defined at:
  File "/home/iplab/anaconda2/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/iplab/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/iplab/anaconda2/lib/python2.7/site-packages/ipykernel/kernelap

In [36]:
#Testing with gated model
def non_ref_img_denoise_metric(noisy_img,denoised_img):
    mni = noisy_img - denoised_img
    ssim_n = compare_ssim(noisy_img,mni,multichannel=True)
    ssim_p = compare_ssim(noisy_img,denoised_img,multichannel=True)
    return pearsonr(ssim_n,ssim_p)


In [92]:
sym_autoencoder = load_model('./models/gated_oct_epoch_400_batch_8_sgd/gated_cnn_autoencoder_oct.hdf5')


In [66]:
def read_images_from_folder_with_names(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path,filename),0)
        if img is not None:
            images.append((filename,img))
    return images

In [67]:
test_image_set = read_images_from_folder_with_names('./test_data/cropped_noisy/')

In [97]:
gt = cv2.imread('test_data/cropped5.tif',0)
gt = np.array(gt)
gt.shape
gt = cv2.resize(gt,(896,288),interpolation = cv2.INTER_CUBIC)
gt =[gt]
gt = np.array(gt,dtype=np.float32)
gt = gt/255.
gt = gt.reshape((288,896,1))
cv2.imwrite("gt.tif",gt*255.)
gt.shape

(288, 896, 1)

In [93]:
for idx,test_image in enumerate(test_image_set):
    test_image_norm = cv2.resize(test_image[1],(896,288),interpolation = cv2.INTER_CUBIC)
    test_image_norm = [test_image_norm]
    test_image_norm = np.array(test_image_norm,dtype=np.float32)
    test_image_norm = test_image_norm/255.
    print (test_image_norm.shape)
    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    print (test_image_norm.shape)
    out_image = sym_autoencoder.predict(test_image_norm,verbose=1)
    #print (out_image[0].shape,gt.shape)
    psnr = compare_psnr(gt*255.,out_image[0]*255.,data_range=256)
    ssim = compare_ssim(gt,out_image[0],multichannel=True)
    print test_image[0][:-3]
    print psnr,ssim
   
    #cv2.imwrite('./test_data/cropped_denoised_tanh/'+test_image[0][:-3] + '_'+str(round(psnr,4))+'_'+str(round(ssim,5))+'.tif',out_image[0]*255.)
    print test_image_norm.shape

(1, 288, 896)
(1, 288, 896, 1)
1/1 [==============================] - 2s 2s/step
oct_shape_8_scale_7.
21.7921047164 0.834842486071
(1, 288, 896, 1)
(1, 288, 896)
(1, 288, 896, 1)
1/1 [==============================] - 1s 1s/step
oct_shape_6_scale_6.
27.3649185447 0.89871801907
(1, 288, 896, 1)
(1, 288, 896)
(1, 288, 896, 1)
1/1 [==============================] - 1s 1s/step
oct_shape_4_scale_3.
28.6942525374 0.941862508866
(1, 288, 896, 1)
(1, 288, 896)
(1, 288, 896, 1)
1/1 [==============================] - 1s 1s/step
oct_shape_4_scale_4.
29.4917949058 0.937623126442
(1, 288, 896, 1)
(1, 288, 896)
(1, 288, 896, 1)
1/1 [==============================] - 1s 1s/step
oct_shape_3_scale_2.
27.1735782581 0.942648639919
(1, 288, 896, 1)


In [32]:
#Testing with simple model

In [56]:
model = load_model("./models/oct_epoch_500_batch_20/simple_cnn_oct.hdf5")

In [19]:
#Adding Gamma noise
test_image = cv2.imread('./test_data/cropped5.tif',0)
print test_image.shape
#test_image = cv2.resize(test_image,(900,448),interpolation = cv2.INTER_CUBIC)
#print test_image.shape
gamma_param = [(3,2),(4,3),(4,4),(6,6),(8,7)]
for param in gamma_param:
    shape = param[0]
    scale = param[1]
    noise = np.random.gamma(shape,scale,test_image.shape)
    noisy_img = np.clip(noise + test_image,0.,255.)
    cv2.imwrite('./test_data/cropped_noisy/oct_shape_'+str(shape)+'_scale_'+str(scale)+'.tif',noisy_img)

(280, 900)


In [58]:
test_image_set = read_images_from_folder_with_names('./test_data/cropped_noisy/')

In [59]:
for idx,test_image in enumerate(test_image_set):
    #test_image_norm = cv2.resize(test_image,(256,128),interpolation = cv2.INTER_CUBIC)
    test_image_norm = [test_image[1]]
    test_image_norm = np.array(test_image_norm,dtype=np.float32)
    test_image_norm = test_image_norm/255.
    test_image_norm = test_image_norm.reshape((test_image_norm.shape[0],test_image_norm.shape[1],test_image_norm.shape[2],1))
    
    out_image = model.predict(test_image_norm,verbose=1)
    print test_image_norm[0].shape,out_image[0].shape
    psnr = compare_psnr(test_image_norm[0]*255.,out_image[0]*255.,data_range=256)
    ssim = compare_ssim(test_image_norm[0],out_image[0],multichannel=True)
    print psnr,ssim
    print test_image[0][:-3]
    cv2.imwrite('./test_data/cropped_denoised/'+test_image[0][:-3] + '_'+str(round(psnr,4))+'_'+str(round(ssim,5))+'.tif',out_image[0]*255.)
    print test_image_norm.shape

1/1 [==============================] - 1s
(280, 900, 1) (280, 900, 1)
11.8148960657 0.39612636236
oct_shape_8_scale_7.
(1, 280, 900, 1)
1/1 [==============================] - 0s
(280, 900, 1) (280, 900, 1)
14.9024515031 0.528854573747
oct_shape_6_scale_6.
(1, 280, 900, 1)
1/1 [==============================] - 0s
(280, 900, 1) (280, 900, 1)
22.8196762881 0.825083118754
oct_shape_4_scale_3.
(1, 280, 900, 1)
1/1 [==============================] - 0s
(280, 900, 1) (280, 900, 1)
20.6693131506 0.759466202918
oct_shape_4_scale_4.
(1, 280, 900, 1)
1/1 [==============================] - 0s
(280, 900, 1) (280, 900, 1)
26.0182223411 0.888059282969
oct_shape_3_scale_2.
(1, 280, 900, 1)


In [60]:
out_image = model.predict(test_image_norm,verbose=1)

1/1 [==============================] - 0s


In [5]:
tf.pack

AttributeError: 'module' object has no attribute 'pack'